### Training code for dataset Bla Bla

Description of the dataset, experiment etc

#### General imports

In [1]:
from typing import Callable, Optional

import wandb
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from careamics.lightning import VAEModule

# Temporary hack
import sys
sys.path.insert(0, "/home/igor.zubarev/projects/microSplit-reproducibility/")

import configs

from configs.factory import (
    get_algorithm_config,
    get_likelihood_config,
    get_loss_config,
    get_model_config,
    get_optimizer_config,
    get_training_config,
    get_lr_scheduler_config,
)
from datasets import create_train_val_datasets

#### Experiments specific imports

In [2]:
from configs.parameters.pavia_medium_balanced import get_denoisplit_parameters
from configs.data.pavia3 import get_data_configs
from datasets.pavia3 import get_train_val_data

### Get configs

Example training code 5 epochs, switch between full training, short training,  fine-tuning 

In [3]:
# TODO refactor, wrap in one function. Only if the function can be generalized
train_data_config, val_data_config, test_data_configs = get_data_configs()
training_params = get_denoisplit_parameters()


In [4]:
for k in train_data_config:
    print(k)    

('data_type', <DataType.Pavia3SeqData: 2>)
('depth3D', 1)
('datasplit_type', <DataSplitType.Train: 1>)
('num_channels', 2)
('ch1_fname', None)
('ch2_fname', None)
('ch_input_fname', None)
('input_is_sum', False)
('input_idx', None)
('target_idx_list', None)
('start_alpha', None)
('end_alpha', None)
('image_size', (64, 64))
('grid_size', 32)
('empty_patch_replacement_enabled', False)
('empty_patch_replacement_channel_idx', None)
('empty_patch_replacement_probab', None)
('empty_patch_max_val_threshold', None)
('uncorrelated_channels', False)
('uncorrelated_channel_probab', 0.5)
('poisson_noise_factor', -1.0)
('synthetic_gaussian_scale', None)
('input_has_dependant_noise', True)
('enable_gaussian_noise', False)
('allow_generation', False)
('training_validtarget_fraction', None)
('deterministic_grid', None)
('enable_rotation_aug', False)
('max_val', None)
('overlapping_padding_kwargs', {'mode': 'reflect'})
('print_vars', False)
('normalized_input', True)
('use_one_mu_std', True)
('train_au

In [5]:
training_params

{'algorithm': 'denoisplit',
 'loss_type': 'denoisplit_musplit',
 'img_size': (64, 64),
 'target_channels': 2,
 'multiscale_count': 3,
 'predict_logvar': 'pixelwise',
 'nm_paths': ['/group/jug/ashesh/training/noise_model/2404/94/GMMNoiseModel_pavia3_sequential_singlefiles-Cond_1__6_4_Clip0.0-1.0_Sig0.125_UpNone_Norm0_bootstrap.npz',
  '/group/jug/ashesh/training/noise_model/2404/97/GMMNoiseModel_pavia3_sequential_singlefiles-Cond_1__6_4_Clip0.0-1.0_Sig0.125_UpNone_Norm0_bootstrap.npz'],
 'kl_type': 'kl_restricted',
 'batch_size': 32,
 'lr': 0.001,
 'lr_scheduler_patience': 30,
 'earlystop_patience': 200,
 'num_epochs': 400,
 'num_workers': 0,
 'mmse_count': 10,
 'grid_size': 32}

### Create dataset

In [6]:
data_path = "/localscratch/data/pavia3_sequential_cropped" # TODO should it be here or in the config?

train_dset, val_dset, _, data_stats = create_train_val_datasets(
    datapath=data_path,
    train_config=train_data_config,
    val_config=val_data_config,
    test_config=val_data_config,
    load_data_func=get_train_val_data,
)

# TODO problem is, creating a dataloader requires a config, that's ugly af


Explicit datasplit Train [1, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16]
Loaded from SubDsetType.MultiChannel /localscratch/data/pavia3_sequential_cropped 13/17 frames

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style

Padding is not used with this alignement style
MultiFileDset avg height: 1288, avg width: 1092, count: 13

Explicit datasplit Val [2, 12]
Loaded from SubDsetType.MultiChannel /localscratch/data/pavia3_sequential_cropped 2/17 frames

Padding is not used with

In [7]:
loss_config = get_loss_config(**training_params)


In [8]:
training_params["data_stats"] = data_stats # TODO rethink

loss_config = get_loss_config(**training_params)
model_config = get_model_config(**training_params)
gaussian_lik_config, noise_model_config, nm_lik_config = get_likelihood_config(
    **training_params
)
training_config = get_training_config(**training_params)
lr_scheduler_config = get_lr_scheduler_config(**training_params)
optimizer_config = get_optimizer_config(**training_params)

# TODO rename to create
experiment_config = get_algorithm_config(
    algorithm=training_params["algorithm"],
    loss_config=loss_config,
    model_config=model_config,
    gaussian_lik_config=gaussian_lik_config,
    nm_config=noise_model_config,
    nm_lik_config=nm_lik_config,
    lr_scheduler_config=lr_scheduler_config,
    optimizer_config=optimizer_config,
)

/home/igor.zubarev/projects/careamics/src/careamics/utils/serializers.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(lst)


### Visualize configs

In [9]:
#TODO code, discuss

### Train the model

Only 5 epochs for the sake of the example

### Create dataloaders

In [9]:
train_dloader = DataLoader(
    train_dset,
    batch_size=training_params["batch_size"],
    num_workers=training_params["num_workers"],
    shuffle=True,
)
val_dloader = DataLoader(
    val_dset,
    batch_size=training_params["batch_size"],
    num_workers=training_params["num_workers"],
    shuffle=False,
)

In [14]:
# init lightning model
lightning_model = VAEModule(algorithm_config=experiment_config)

# train the model
# custom_callbacks = get_callbacks(logdir)


[GaussianMixtureNoiseModel] min_sigma: 0.125
[GaussianMixtureNoiseModel] min_sigma: 0.125
[MultiChannelNoiseModel] Nmodels count:2
[GaussianLikelihood] PredLVar:pixelwise LowBLVar:-5.0


### Training logs

In [17]:
trainer = Trainer(
    max_epochs=training_config.num_epochs,
    accelerator="gpu",
    enable_progress_bar=True,
    # callbacks=custom_callbacks,
    precision=training_config.precision,
    gradient_clip_val=training_config.gradient_clip_val,
    gradient_clip_algorithm=training_config.gradient_clip_algorithm,
    # num_sanity_val_steps=0
)
trainer.fit(
    model=lightning_model,
        train_dataloaders=train_dloader,
        val_dataloaders=val_dloader,
    )

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                   | Params | Mode
-------------------------------------------------------------------------
0 | model                  | LadderVAE              | 3.6 M  | eval
1 | noise_model            | MultiChannelNoiseModel | 0      | eval
2 | noise_model_likelihood | NoiseModelLikelihood   | 0      | eval
3 | gaussian_likelihood    | GaussianLikelihood     | 0      | eval
-------------------------------------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.270    Total estimated model params size (MB)
0         Modules in train mode
321       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/localscratch/mamba/envs/splits/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/localscratch/mamba/envs/splits/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 0:   2%|▏         | 13/592 [00:05<03:53,  2.48it/s, v_num=18]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# TODO visualize losses, CSV logger as pandas df, plots, location of checkpoint 